In [ ]:
import numpy as np
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, Input, Reshape, Flatten, \
                         Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.models import Model, save_model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
shape = (288, 288, 3)
batch_size = 32
epochs = 15
path_train = r'/content/drive/MyDrive/DS/train'
path_val = r'/content/drive/MyDrive/DS/test'
train_generator = ImageDataGenerator(rescale=1./255,)
val_generator = ImageDataGenerator(rescale=1./255,)

In [ ]:
train_gen = train_generator.flow_from_directory(
    directory = path_train,
    target_size = (shape[0], shape[1]),
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'input',
    shuffle = True,
    seed = 42,
    )

val_gen = val_generator.flow_from_directory(
    directory = path_val,
    target_size = (shape[0], shape[1]),
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'input',
    shuffle = True,
    seed = 42,
    )

Found 8000 images belonging to 1 classes.
Found 2000 images belonging to 1 classes.


In [ ]:
# Encoder
input_img = Input(shape = shape)
x = Conv2D(16, (3, 3), activation = 'relu', padding = 'same')(input_img)
x = MaxPooling2D((2, 2), padding = 'same')(x)
x = Conv2D(8, (3, 3), activation = 'relu', padding = 'same')(x)
x = MaxPooling2D((2, 2), padding = 'same')(x)
x = Conv2D(8, (3, 3), activation = 'relu', padding = 'same')(x)
encoded = MaxPooling2D((2, 2), padding = 'same')(x)

# Decoder
x = Conv2DTranspose(8, (3, 3), activation = 'relu', padding = 'same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(8, (3, 3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation = 'relu', padding = 'same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation = 'sigmoid', padding = 'same')(x)

# Autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer = 'adam', loss = 'binary_crossentropy')

# Print the model summary
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 288, 288, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 288, 288, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 144, 144, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 144, 144, 8)       1160      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 72, 72, 8)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 72, 72, 8)         584   

In [ ]:
# Train the autoencoder using the generator
autoencoder.fit(
    train_gen,
    epochs = epochs,
    validation_data = (val_gen),
    batch_size = 32
)

autoencoder.save(r'/content/drive/MyDrive/autoencoder.h5')

Epoch 1/15


InvalidArgumentError: ignored